# Oasis FM Testing Tool

This notebook allows example insurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included

The following terminology is defined in OED. Please visit https://github.com/Simplitium/OED for more information.



## Input files 

Files must be named the following:

* account.csv
* location.csv
* ri_info.csv
* ri_scope.csv


In [ ]:
# Notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os
import shutil



# 3rd party Python libraries
import fm_testing_tool.widgets as widgets
import fm_testing_tool.functions as functions
from IPython.display import Image
import pandas as pd
from IPython.display import Markdown, display, Javascript
from oasislmf.manager import OasisManager



In [ ]:
# Upload local files into the Notebook (Optional)

widgets.file_uploader('./validation/examples/uploaded')

In [ ]:
# Select a validation example to run, the default value points the file upload location
# If a variables shows a 'None' results then the s
source_exposure = {}
widgets.select_source_dir(source_exposure, examples_dir='./validation/examples')

In [ ]:
# Running this cell will execute the entire notebook 
display(Javascript("Jupyter.notebook.execute_cells_below()"))

## Set Test Parameters 

In [ ]:
## Edit test Params 
number_of_subperils = 1
supported_coverage_types = [1,2] # 1 = buildings, 2 = other, 3 = content, 4 = business interruption
loss_factors = [1, 0.6]
alloc_il = 2
alloc_ri = 3
output_level = 'loc' # valid options are 'item', 'loc', 'pol', 'acc', 'port'

In [ ]:
# Load source files from selected test case 
location_df =  functions.load_df(source_exposure['location_path'], required_file='location.csv')
account_df =  functions.load_df(source_exposure['account_path'], required_file='account.csv')
ri_info_df =  functions.load_df(source_exposure['ri_info_path'])
ri_scope_df =  functions.load_df(source_exposure['ri_scope_path'])

In [ ]:
# View/edit the location data.
location_grid = widgets.show_df(location_df)
location_grid

In [ ]:
# View/edit the account data. 
account_grid = widgets.show_df(account_df)
account_grid

In [ ]:
# View/edit the ri_info data (Optinal). 
if ri_info_df is not None:
    ri_info_grid = widgets.show_df(ri_info_df)
    ri_info_grid

In [ ]:
# View/edit the ri_scope data (Optinal). 
if ri_scope_df is not None:
    ri_scope_grid = widgets.show_df(ri_scope_df)
    ri_scope_grid

In [ ]:
# Store exposure and create run dir 
run_dir = os.path.join('runs', os.path.basename(source_exposure['source_dir']))
os.makedirs(run_dir, exist_ok=True)

# Pick up any edits for required files in the grid before running the analysis
location_df = location_grid.get_changed_df()
loc_csv = os.path.join(run_dir, "location.csv")
location_df.to_csv(path_or_buf=loc_csv, encoding='utf-8', index=False)

account_df = account_grid.get_changed_df()
acc_csv = os.path.join(run_dir, "account.csv")
account_df.to_csv(path_or_buf=acc_csv, encoding='utf-8', index=False)
 

# Pick up any edits ri file edits if there 
if (ri_scope_df is not None) and (ri_info_df is not None):
    ri_info_df = ri_info_grid.get_changed_df()
    info_csv = os.path.join(run_dir, "ri_info.csv")
    ri_info_df.to_csv(path_or_buf=info_csv, encoding='utf-8', index=False)
    
    ri_scope_df = ri_scope_grid.get_changed_df()
    scope_csv = os.path.join(run_dir, "ri_scope.csv")
    ri_scope_df.to_csv(path_or_buf=scope_csv, encoding='utf-8', index=False)
else:
    info_csv = None
    scope_csv = None

In [ ]:
# Generate keys file
keys_csv = os.path.join(run_dir, "keys.csv")
OasisManager().generate_keys_deterministic(
    oed_location_csv=loc_csv,
    keys_data_csv=keys_csv,
    supported_oed_coverage_types=supported_coverage_types,
    num_subperils=number_of_subperils,
)

keys_df = functions.load_df(keys_csv, required_file='keys.csv') 
keys_grid = widgets.show_df(keys_df)
keys_grid

## Generate Oasis files

In [ ]:
## Generate Oasis files 

# Pick up any edits in the Keys data 
keys_df = keys_grid.get_changed_df()
keys_df.to_csv(path_or_buf=keys_csv, encoding='utf-8', index=False)

# Start Oasis files generation
oasis_files = OasisManager().generate_files(
    oasis_files_dir=run_dir,
    oed_location_csv=loc_csv,
    oed_accounts_csv=acc_csv,
    oed_info_csv=info_csv,
    oed_scope_csv=scope_csv,
    keys_data_csv=keys_csv,
    disable_summarise_exposure=True,
    write_ri_tree=True,
)

In [ ]:
# Show FM items 
fm_file = 'items'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


In [ ]:
# Show FM coverages 
fm_file = 'coverages'
display(Markdown(f'### {fm_file}.csv'))
widgets.show_df(functions.load_df(oasis_files[fm_file]))


## Direct Insurance files  

In [ ]:
# Show FM summary map 
fm_file = 'fm_summary_map'
display(Markdown(f'### {fm_file}.csv'))
#widgets.show_df(functions.load_df(os.path.join(run_dir, 'gul_summary_map.csv')))
fm_summary_map = functions.load_df(os.path.join(run_dir, 'fm_summary_map.csv'))
widgets.show_df(fm_summary_map)


In [ ]:
# Show FM fm_programme 
fm_file = 'fm_programme'
display(Markdown(f'### {fm_file}.csv'))
fm_programme = functions.load_df(oasis_files[fm_file])
widgets.show_df(fm_programme)

In [ ]:
# Show FM fm_profile 
fm_file = 'fm_profile'
display(Markdown(f'### {fm_file}.csv'))
fm_profile = functions.load_df(oasis_files[fm_file])
widgets.show_df(fm_profile)

In [ ]:
# Show FM fm_policytc 
fm_file = 'fm_policytc'
display(Markdown(f'### {fm_file}.csv'))
fm_policytc = functions.load_df(oasis_files[fm_file])
widgets.show_df(fm_policytc)

In [ ]:
# Display FM tree
fm_tree = functions.create_fm_tree(fm_programme, fm_policytc, fm_profile, fm_summary_map)
functions.render_fm_tree(fm_tree, filename='tree.png')
display(Markdown(f'### FM calc Tree'))
Image(filename='tree.png') 

## Reinsurance files  

In [ ]:
tree_fp = os.path.join(oasis_files['RI_1'], 'fm_tree.png')
Image(filename=tree_fp)

In [ ]:
ri_programme = functions.load_df(os.path.join(oasis_files['RI_1'], 'fm_programme.csv'))
ri_policytc = functions.load_df(os.path.join(oasis_files['RI_1'], 'fm_policytc.csv'))
ri_profile = functions.load_df(os.path.join(oasis_files['RI_1'], 'fm_profile.csv'))

ri_tree = functions.create_fm_tree(ri_programme, ri_policytc, ri_profile, fm_summary_map)
functions.render_fm_tree(ri_tree, filename='ri_tree.png')
display(Markdown(f'### FM calc Tree'))
Image(filename='ri_tree.png') 

# Generate Losses (Ktools FM)


In [ ]:
%%time
# Run Deterministic Losses FM
output_losses = os.path.join(run_dir, 'losses.csv')
OasisManager().run_exposure(
    src_dir=run_dir,
    output_level=output_level,
    output_file=output_losses,
    num_subperils=number_of_subperils,
    loss_factor=loss_factors,
    ktools_alloc_rule_il=alloc_il,
    ktools_alloc_rule_ri=alloc_ri,
    net_ri=True,
    include_loss_factor=True,
    print_summary=False,
    fmpy=False,
)
ktools_losses = functions.load_df(output_losses)



In [ ]:
if len(loss_factors) > 1:
    for i in range(len(loss_factors)):
        factor_losses = ktools_losses[ktools_losses.loss_factor_idx == i]
        display(Markdown(f'### Loss Factor {loss_factors[i]*100} %'))

        display(Markdown('**Total gul** = {}'.format(
            factor_losses.loss_gul.sum()
        )))
        display(Markdown('**Total il** = {}'.format(
            factor_losses.loss_il.sum()
        )))
        if hasattr(factor_losses, 'loss_ri'):
            display(Markdown('**Total ri ceded** = {}'.format(
                factor_losses.loss_ri.sum()
            )))
        
        display(factor_losses.drop(columns='loss_factor_idx'))
else: 
    display(ktools_losses)

# Generate Losses (Python FM)

In [ ]:
%%time
# Run Deterministic Losses fmpy
output_losses = os.path.join(run_dir, 'losses.csv')
OasisManager().run_exposure(
    src_dir=run_dir,
    output_level=output_level,
    output_file=output_losses,
    num_subperils=number_of_subperils,
    loss_factor=loss_factors,
    ktools_alloc_rule_il=alloc_il,
    ktools_alloc_rule_ri=alloc_ri,
    net_ri=True,
    include_loss_factor=True,
    print_summary=False,
    fmpy=True,
)
fmpy_losses = functions.load_df(output_losses)


In [ ]:
if len(loss_factors) > 1:
    for i in range(len(loss_factors)):
        factor_losses = fmpy_losses[fmpy_losses.loss_factor_idx == i]
        display(Markdown(f'### Loss Factor {loss_factors[i]*100} %'))

        display(Markdown('**Total gul** = {}'.format(
            factor_losses.loss_gul.sum()
        )))
        display(Markdown('**Total il** = {}'.format(
            factor_losses.loss_il.sum()
        )))
        if hasattr(factor_losses, 'loss_ri'):
            display(Markdown('**Total ri ceded** = {}'.format(
                factor_losses.loss_ri.sum()
            )))
            
        display(factor_losses.drop(columns='loss_factor_idx'))
else: 
    display(fmpy_losses)

## Percentage difference between FM modules 

In [ ]:
# Show output difference % 
if len(loss_factors) > 1:
    pc_diff_df = fmpy_losses[['portnumber', 'accnumber','locnumber', 'loss_factor_idx']]
    pc_diff_df['loss_il_difference'] = (fmpy_losses['loss_il'] - ktools_losses['loss_il']) / (fmpy_losses['loss_il'] + ktools_losses['loss_il'] * 2)* 100
    if hasattr(fmpy_losses, 'loss_ri'):
        pc_diff_df['loss_ri_difference'] = (fmpy_losses['loss_ri'] - ktools_losses['loss_ri']) / (fmpy_losses['loss_ri'] + ktools_losses['loss_ri'] * 2)* 100
    
    pc_diff_df.fillna(0, inplace=True)
    
    for i in range(len(loss_factors)):
        display(Markdown(f'### Loss Factor {loss_factors[i]*100} %'))
        diff_losses = pc_diff_df[pc_diff_df.loss_factor_idx == i]
        display(diff_losses.drop(columns='loss_factor_idx'))
    
else: 
    pc_diff_df = fmpy_losses[['portnumber', 'accnumber','locnumber']]
    pc_diff_df['loss_il_difference'] = (fmpy_losses['loss_il'] - ktools_losses['loss_il']) / (fmpy_losses['loss_il'] + ktools_losses['loss_il'] * 2)* 100
    pc_diff_df['loss_ri_difference'] = (fmpy_losses['loss_ri'] - ktools_losses['loss_ri']) / (fmpy_losses['loss_ri'] + ktools_losses['loss_ri'] * 2)* 100
    pc_diff_df.fillna(0, inplace=True)
    display(pc_diff_df)